(https://www.kaggle.com/c/ashrae-energy-prediction/)

# ASRAE - Modeling

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import plotly as plt
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

from math import sqrt

import glob

import pandas_profiling as ppf
import sweetviz as sv
import missingno as msno

import pprint
import datetime
from math import sqrt

from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, train_test_split

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm

from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

from prophet import Prophet

import xgboost as xgb


import pmdarima as pm
from pmdarima import model_selection
from pmdarima import pipeline
from pmdarima import preprocessing as ppc
from pmdarima import arima
from pmdarima.arima import ndiffs
from pmdarima.arima import ADFTest

print("pmdarima version: %s" % pm.__version__)

import tools as tl

sns.set(rc={'figure.figsize':(30,20)})

pd.options.display.float_format = '{:.2f}'.format

plt.rcParams.update({'font.size': 22})
sns.set(font_scale = 2)

import pprint
pp = pprint.PrettyPrinter(indent=4)

pmdarima version: 1.8.2


In [2]:
print(xgb.__version__)

1.4.2


In [3]:
pd.options.plotting.backend = "matplotlib"

In [4]:
pwd

'C:\\Users\\Mike\\Google Drive\\Code\\Springboard_Capstone_Energy'

## See Statistics written for each file (building-meter) combo

In [46]:
# See Building Statistics for missing data
df_stats = pd.read_csv('.//data_clean//building_reading_stats.csv')
# Keep only data that has at least 50 % of it


In [14]:
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1767 entries, 0 to 1766
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   file_name                     1767 non-null   object 
 1   building                      1767 non-null   int64  
 2   meter                         1767 non-null   int64  
 3   len_train                     1767 non-null   int64  
 4   len_zero                      1767 non-null   int64  
 5   per_data                      1767 non-null   float64
 6   category                      1767 non-null   object 
 7   corr_meter_to_airtemp         1767 non-null   float64
 8   corr_meter_to_airtemp_wknd    1766 non-null   float64
 9   corr_meter_to_airtemp_wkdy    1767 non-null   float64
 10  corr_meter_to_airtemp_winter  1767 non-null   float64
 11  corr_meter_to_airtemp_spring  1755 non-null   float64
 12  corr_meter_to_airtemp_summer  1759 non-null   float64
 13  cor

In [15]:
# Modeling Choices
# Buildings with 1

In [19]:
# These represent buildings that are in top, mid and lower in square footage
lrg = [869, 1148, 375, 365, 269]
med = [1063, 1309, 991, 144, 693]
sm = [846, 621, 822, 816, 44]

bldgno = []
bldgno.extend(lrg)
bldgno.extend(med)
bldgno.extend(sm)

# 0: Electricity 1: chilledwater 2: steam 3: hotwater

In [20]:
df_stats[(df_stats['building'] == 1148) & df_stats['meter'] == 1]

,file_name,building,meter,len_train,len_zero,per_data,category,corr_meter_to_airtemp,corr_meter_to_airtemp_wknd,corr_meter_to_airtemp_wkdy,corr_meter_to_airtemp_winter,corr_meter_to_airtemp_spring,corr_meter_to_airtemp_summer,corr_meter_to_airtemp_fall,square_feet,floor_count
194,1148_1.pkl,1148,1,8784,0,100.00,Office,0.78,0.79,0.79,0.21,0.74,0.45,0.76,861524,-1


In [21]:
# Examine building choices
df_stats[df_stats['building'] == 1148]

,file_name,building,meter,len_train,len_zero,per_data,category,corr_meter_to_airtemp,corr_meter_to_airtemp_wknd,corr_meter_to_airtemp_wkdy,corr_meter_to_airtemp_winter,corr_meter_to_airtemp_spring,corr_meter_to_airtemp_summer,corr_meter_to_airtemp_fall,square_feet,floor_count
193,1148_0.pkl,1148,0,8663,0,100.00,Office,-0.02,0.15,-0.04,-0.20,-0.11,-0.08,-0.08,861524,-1
194,1148_1.pkl,1148,1,8784,0,100.00,Office,0.78,0.79,0.79,0.21,0.74,0.45,0.76,861524,-1
902,1148_2.pkl,1148,2,8784,2,99.98,Office,-0.88,-0.86,-0.88,-0.83,-0.85,-0.43,-0.44,861524,-1


### Build list of files to include

In [ ]:
# Get only the train data
direct = glob.glob('.\\data_clean\\buildings_by_meter\\' + '*.pkl')

directory = '.\\data_clean\\buildings_by_meter\\'
ext = '.pkl'

bld_mtr = ['1148_0','1148_1']
files = []

# Build File names
for bm in bld_mtr:
    file_n = directory + bm + ext
    files.append(file_n)
        
files

In [ ]:
files = []
direct = '.\\data_clean\\buildings_by_meter\\'
df_train = pd.read_pickle(direct + file)

# Build 3 Models

In [85]:
# Holes in the data
# Data is daily
offsets = [0] # offsets in days
data_sizes = [90,120]
test_sizes = [int(x * 0.25) for x in data_sizes]

ARIMA = True
FBProphet = True
xgboost = True

quiet_mode = True

resample = True
backfill = 'mean'
exogen = True

thresh = 50

# For each of the file matches (did this way to prevent long run times)

# Dictionary to hold all the results, will turn into a dataframe
stats_all = {}
stats_list = []

# For each file in the directory
direct = '.\\data_clean\\buildings_by_meter\\'

files = df_stats['file_name']
bl = df_stats['building']

for num,file in enumerate(files):

    # Termporary dictionary that represents a row
    stats = {}
    
    # Skip if building not on our radar
    if bl[num] not in bldgno:
        continue
    else:
        print('File: ' + file)
        df_train = pd.read_pickle(direct + file)
     
    # Load key variables in dictionary
    stats['building'] = df_train.iloc[0]['building_id']
    stats['square_feet'] = df_train.iloc[0]['square_feet']
    
    stats['meter'] = bldg_meter = df_train.iloc[0]['meter']
    stats['file_name'] = file

#     df_train = pd.get_dummies(df_train)
#     print(df_train)
    
    # RESAMPLE DATA
    if resample:
        agg_dict={'meter_reading':'sum', 
                  'air_temperature':'max',
                  'dew_temperature':'max',
                  'cloud_coverage':'mean',
                  'building_id': 'max',
                  'site_id': 'max',
                  'square_feet': 'max',
                  'day_of_week':'max',
                  'weekend':'max',
                  'month':'max',
                  'season':'max'
                  }

        resamp = '1D'
        
        # Write Resample String
        stats['resample_period'] = resamp

        # Resample Dataframe
        df_train = df_train.resample(resamp).agg(agg_dict)
        
        # Drops
#         print(df_train.head())
        
    # Keep track of Models
    model_ctr = 1
    model_prefixes = []
    
    # Pull only training data
    
    # Curently for 4 periods data sizes is length, test_sizes is 0.25 % of that
    for ds, ts in zip(data_sizes,test_sizes):     
        for off in offsets:
            # df_slice is the total size of the set
            
            df_slice = df_train.iloc[off:off+ds]
            
            df_slice.dropna(subset = ['meter_reading', 'air_temperature'],how='any')
            
#             if df_slice['sea_level_pressure'].iloc[0]:
#                 df_slice.drop(labels='sea_level_pressure',inplace=True,axis=1)
            
#             for column in df_slice.columns:
#                 # Percent of data that is not zero
#                 z_per = 100-((df_slice[column] == 0).sum()/len(df_slice[column])*100)
#                 n_per = 100-((df_slice[column].isna()).sum()/len(df_slice[column])*100)
                
#             print(df_slice.count(axis=1))
    
            if ARIMA:
                # Keep track of model prefix column names by building prefix to number iterations
                model_prefix = 'Model_' + str(model_ctr)
                model_prefixes.append(model_prefix)

                #df_slice.dropna(inplace=True)
                X_train, X_test, y_train, y_test = model_selection.train_test_split(df_slice.drop(labels=['meter_reading'],axis=1),
                                                                                    df_slice['meter_reading'],test_size=ts)    
                # Fill Nas
                #Counts of null values 
                # Backfill with means
                if np.max(X_train.isnull().sum().sort_values(ascending=False)) > 0:
                    stats[model_prefix + '_' + 'X_train_Backfill'] = backfill
                    if backfill == 'mean':
                        X_train.fillna(X_train.mean(), inplace=True)

                # Backfill with mean
                if np.max(X_test.isnull().sum().sort_values(ascending=False)) > 0:
                    stats[model_prefix + '_' + 'X_test_Backfill'] = backfill
                    if backfill == 'mean':
                        X_test.fillna(X_test.mean(), inplace=True)

                # Get rid of constants
                X_train.drop(labels=['site_id','building_id','square_feet'],axis=1,inplace=True)
                X_test.drop(labels=['site_id','building_id','square_feet'],axis=1,inplace=True)

                # Populate DataFrame with statistics
                stats[model_prefix + '_' + 'Type'] = 'ARIMA_Exogen'
                stats[model_prefix + '_' + 'DateStart'] = str(df_slice.iloc[0].name).split(" ")[0]
                stats[model_prefix + '_' + 'DateEnd'] = str(df_slice.iloc[-1].name).split(" ")[0]
                stats[model_prefix + '_' + 'DayOffset'] = off
                stats[model_prefix + '_' + 'DataSize'] = ds
                stats[model_prefix + '_' + 'TestSize'] = len(y_test)
                stats[model_prefix + '_' + 'X_train_len'] = len(X_train)
                stats[model_prefix + '_' + 'X_test_len'] = len(X_test)
                stats[model_prefix + '_' + 'y_train_len'] = len(y_train)
                stats[model_prefix + '_' + 'y_test_len'] = len(y_test)

   #           ######################################    ARIMA   ##################################################
                print("\n" * 1)
                print('ARIMA Model......',end=' ')
                
                try:
                    model = pm.auto_arima(y_train,X=X_train,trace=False,n_fits=10,seasonal=True,m=7,error_action='ignore')
                    stats[model_prefix + '_' + 'Status'] = 'PASS'
                except ValueError as ve:
                    print('Model will not converge')
                    stats[model_prefix + '_' + 'Status'] = 'FAIL'
                    
                    
                preds, conf_int = model.predict(X=X_test,n_periods=y_test.shape[0], return_conf_int=True)

                predict = pd.Series(preds,index=y_test.index)

                # Keep each slice data here
                df_result = pd.concat([predict,y_test],axis=1)
                df_result.columns = ['meter_predict_ARIMA','meter_actual']

                RMSE_error = (mean_squared_error(df_result['meter_predict_ARIMA'],df_result['meter_actual'],squared=False))

                stats[model_prefix + '_' + 'RMSE'] = RMSE_error
                stats[model_prefix + '_' + 'y_test_MAX'] = np.max(df_result['meter_actual'])
                stats[model_prefix + '_' + 'y_test_MIN'] = np.min(df_result['meter_actual'])
                stats[model_prefix + '_' + 'y_test_AVG'] = np.mean(df_result['meter_actual'])
                
                stats[model_prefix + '_' + 'y_pred_MAX'] = np.max(df_result['meter_predict_ARIMA'])
                stats[model_prefix + '_' + 'y_pred_MIN'] = np.min(df_result['meter_predict_ARIMA'])
                stats[model_prefix + '_' + 'y_pred_AVG'] = np.mean(df_result['meter_predict_ARIMA'])
                
                if not quiet_mode:
                    # display
                    plt.figure(figsize=(20,10))
                    plt.plot(df_result['meter_predict_ARIMA'],label="ARIMA")
                    plt.plot(df_result['meter_actual'],label="Actual")
                    plt.legend(loc = 'upper right')
                    plt.savefig('SecondPrection.jpg')
                    plt.show()
                    
                    print('RMSE: ' + str(RMSE_error))
                    


            
            ######################################    PROPHET   ##################################################
            if FBProphet:
                print("\n" * 1)
                print('Prophet......')
                model_ctr += 1
                model_prefix = 'Model_' + str(model_ctr)
                model_prefixes.append(model_prefix)

                # Prophet keeps target variable with regressors
                df_train, df_test = model_selection.train_test_split(df_slice,test_size=ts)

                # Need renaming for Prophet - data prep
                df_prophet_train = df_train.reset_index()
                df_prophet_test = df_test.reset_index()

                # Fill Nas
                #Counts of null values 
                # Backfill with means
                if np.max(df_prophet_train.isnull().sum().sort_values(ascending=False)) > 0:
                    stats[model_prefix + '_' + 'train_Backfill'] = backfill
                    if backfill == 'mean':
                        df_prophet_train.fillna(df_prophet_train.mean(), inplace=True)

                # Backfill with mean
                if np.max(df_prophet_test.isnull().sum().sort_values(ascending=False)) > 0:
                    stats[model_prefix + '_' + 'test_Backfill'] = backfill
                    if backfill == 'mean':
                        df_prophet_test.fillna(df_prophet_test.mean(), inplace=True)


                # Timestamp must be ds column
                df_prophet_train = df_prophet_train.rename(columns={'timestamp':'ds','meter_reading':'y'})
                df_prophet_test = df_prophet_test.rename(columns={'timestamp':'ds','meter_reading':'y'})


                stats[model_prefix + '_' + 'Type'] = 'Prophet_Exogen'
                stats[model_prefix + '_' + 'DateStart'] = str(df_prophet_train.iloc[0]['ds']).split(" ")[0]
                stats[model_prefix + '_' + 'DateEnd'] = str(df_prophet_train.iloc[-1]['ds']).split(" ")[0]
                stats[model_prefix + '_' + 'DayOffset'] = off
                stats[model_prefix + '_' + 'DataSize'] = ds
                stats[model_prefix + '_' + 'Train_len'] = len(df_prophet_train)
                stats[model_prefix + '_' + 'Test_len'] = len(df_prophet_test)
    #           print(df_prophet_train)

                m = Prophet(daily_seasonality=True)
                m.add_regressor('air_temperature')
#                 m.add_regressor('dew_temperature')
#                 m.add_regressor('cloud_coverage')
#                 m.add_regressor('precip_depth_1_hr')
                m.add_regressor('day_of_week')
                m.add_regressor('weekend')
                m.add_regressor('month')
                m.add_regressor('season')
                m.fit(df_prophet_train)
                predicts = m.predict(df_prophet_test)

                RMSE_error = (mean_squared_error(predicts['yhat'],df_prophet_test['y'],squared=False))

                stats[model_prefix + '_' + 'RMSE'] = RMSE_error
                stats[model_prefix + '_' + 'y_test_MAX'] = np.max(df_prophet_test['y'])
                stats[model_prefix + '_' + 'y_test_MIN'] = np.min(df_prophet_test['y'])
                stats[model_prefix + '_' + 'y_test_AVG'] = np.mean(df_prophet_test['y'])
                
                stats[model_prefix + '_' + 'y_pred_MAX'] = np.max(predicts['yhat'])
                stats[model_prefix + '_' + 'y_pred_MIN'] = np.min(predicts['yhat'])
                stats[model_prefix + '_' + 'y_pred_AVG'] = np.mean(predicts['yhat'])

                if not quiet_mode:
                    plt.figure(figsize=(20,10))
                    plt.plot(predicts['yhat'],label="Prophet")
                    plt.plot(df_prophet_test['y'],label="Actual")
                    plt.legend(loc = 'upper right')
                    plt.show()
                    print('RMSE: ' + str(RMSE_error))
                    stats[model_prefix + '_' + 'Status'] = 'PASS'
     
            ######################################    XGBOOST   ##################################################
            if xgboost:
                print("\n" * 1)
                print('XGBoost......')
                model_ctr += 1
                model_prefix = 'Model_' + str(model_ctr)
                model_prefixes.append(model_prefix)
                
                # split data into X and y
                X = df_slice.drop('meter_reading', axis=1)
                y = df_slice[['meter_reading']]
                
                # Fill Nas
                #Counts of null values 
                # Backfill with means
                if np.max(X.isnull().sum().sort_values(ascending=False)) > 0:
                    stats[model_prefix + '_' + 'train_Backfill'] = backfill
                    X.fillna(X.median(), inplace=True)

                # Train Test Split
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ts)
                        
                stats[model_prefix + '_' + 'Type'] = 'XGBoost'
                stats[model_prefix + '_' + 'DateStart'] = str(df_slice.iloc[0].name).split(" ")[0]
                stats[model_prefix + '_' + 'DateEnd'] = str(df_slice.iloc[-1].name).split(" ")[0]
                stats[model_prefix + '_' + 'DayOffset'] = off
                stats[model_prefix + '_' + 'DataSize'] = ds
                stats[model_prefix + '_' + 'Train_len'] = len(X_train)
                stats[model_prefix + '_' + 'Test_len'] = len(X_test)
                
                
#                 # Instantiate Model
                gbm_model = xgb.XGBRegressor(objective = 'reg:squarederror', n_estimators = 1000, seed = 123)
                gbm_model.fit(X_train,y_train)
                predict = gbm_model.predict(X_test)
                
                predict = pd.Series(predict,index=y_test.index)
                
                df_result = pd.concat([predict,y_test],axis=1,ignore_index=True)
                df_result.columns = ['meter_predict_XGBoost','meter_actual']
                df_result.sort_index(inplace=True)
                
                
                RMSE_error = (mean_squared_error(predict,y_test,squared=False))

                stats[model_prefix + '_' + 'RMSE'] = RMSE_error
                stats[model_prefix + '_' + 'y_test_MAX'] = np.max(y_test['meter_reading'])
                stats[model_prefix + '_' + 'y_test_MIN'] = np.min(y_test['meter_reading'])
                stats[model_prefix + '_' + 'y_test_AVG'] = np.mean(y_test['meter_reading'])
                
                stats[model_prefix + '_' + 'y_pred_MAX'] = np.max(predict)
                stats[model_prefix + '_' + 'y_pred_MIN'] = np.min(predict)
                stats[model_prefix + '_' + 'y_pred_AVG'] = np.mean(predict)
                
                if not quiet_mode:
                    plt.figure(figsize=(20,10))
                    plt.plot(df_result['meter_predict_XGBoost'],label="XGBoost")
                    plt.plot(df_result['meter_actual'],label="Actual")
                    plt.legend(loc = 'upper right')
                    plt.show()
                    print('RMSE: ' + str(RMSE_error))
                    
    stats_list.append(stats)
#     pp.pprint(stats)
df_results = pd.DataFrame(stats_list)


File: 1148_0.pkl


ARIMA Model...... 

<ipython-input-85-9b6a6d8ec6d2>:208: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_prophet_train.fillna(df_prophet_train.mean(), inplace=True)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.




Prophet......


XGBoost......


<ipython-input-85-9b6a6d8ec6d2>:208: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_prophet_train.fillna(df_prophet_train.mean(), inplace=True)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.




ARIMA Model...... 

Prophet......


XGBoost......
File: 1148_1.pkl


ARIMA Model...... 

...



ARIMA Model...... 

Prophet......


XGBoost......
File: 144_3.pkl


ARIMA Model...... 

<ipython-input-85-9b6a6d8ec6d2>:208: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_prophet_train.fillna(df_prophet_train.mean(), inplace=True)
<ipython-input-85-9b6a6d8ec6d2>:214: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_prophet_test.fillna(df_prophet_test.mean(), inplace=True)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.




Prophet......


XGBoost......


<ipython-input-85-9b6a6d8ec6d2>:208: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_prophet_train.fillna(df_prophet_train.mean(), inplace=True)
<ipython-input-85-9b6a6d8ec6d2>:214: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_prophet_test.fillna(df_prophet_test.mean(), inplace=True)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.




ARIMA Model...... 

Prophet......


XGBoost......


In [86]:
df_results.iloc[0]

building                         1148
square_feet                    861524
meter                               0
file_name                  1148_0.pkl
resample_period                    1D
                              ...    
Model_5_y_pred_AVG           22462.84
Model_1_X_test_Backfill           NaN
Model_2_test_Backfill             NaN
Model_3_X_test_Backfill           NaN
Model_4_test_Backfill             NaN
Name: 0, Length: 95, dtype: object

In [87]:
df= df_results.filter(regex='Model_')
df.head()

,Model_1_X_train_Backfill,Model_1_Type,Model_1_DateStart,Model_1_DateEnd,Model_1_DayOffset,Model_1_DataSize,Model_1_TestSize,Model_1_X_train_len,Model_1_X_test_len,Model_1_y_train_len,...,Model_5_y_test_MAX,Model_5_y_test_MIN,Model_5_y_test_AVG,Model_5_y_pred_MAX,Model_5_y_pred_MIN,Model_5_y_pred_AVG,Model_1_X_test_Backfill,Model_2_test_Backfill,Model_3_X_test_Backfill,Model_4_test_Backfill
0,mean,ARIMA_Exogen,2016-01-01,2016-03-30,0,90,22,68,22,68,...,25784.06,0.00,21878.83,26984.74,-0.00,22462.84,NaN,NaN,NaN,NaN
1,NaN,ARIMA_Exogen,2016-01-01,2016-03-30,0,90,22,68,22,68,...,2039.56,906.26,1136.58,2382.04,936.40,1216.25,NaN,NaN,NaN,NaN
2,NaN,ARIMA_Exogen,2016-01-01,2016-03-30,0,90,22,68,22,68,...,66817.92,28008.00,41955.44,49249.32,30647.43,38985.31,NaN,NaN,NaN,NaN
3,mean,ARIMA_Exogen,2016-01-01,2016-03-30,0,90,22,68,22,68,...,26.00,3.00,12.00,15.49,7.81,12.13,mean,mean,NaN,NaN
4,mean,ARIMA_Exogen,2016-01-01,2016-03-30,0,90,22,68,22,68,...,118.00,0.00,84.87,202.00,0.00,90.03,mean,mean,mean,mean


In [88]:
df_stats = df_stats.merge(df_results,how='right',on=['building','meter'])

In [89]:
df_stats

,file_name_x,building,meter,len_train,len_zero,per_data,category,corr_meter_to_airtemp,corr_meter_to_airtemp_wknd,corr_meter_to_airtemp_wkdy,...,Model_5_y_test_MAX,Model_5_y_test_MIN,Model_5_y_test_AVG,Model_5_y_pred_MAX,Model_5_y_pred_MIN,Model_5_y_pred_AVG,Model_1_X_test_Backfill,Model_2_test_Backfill,Model_3_X_test_Backfill,Model_4_test_Backfill
0,1148_0.pkl,1148,0,8663,0,100.00,Office,-0.02,0.15,-0.04,...,25784.06,0.00,21878.83,26984.74,-0.00,22462.84,NaN,NaN,NaN,NaN
1,1148_1.pkl,1148,1,8784,0,100.00,Office,0.78,0.79,0.79,...,2039.56,906.26,1136.58,2382.04,936.40,1216.25,NaN,NaN,NaN,NaN
2,869_0.pkl,869,0,8447,0,100.00,Entertainment/public assembly,-0.22,-0.24,-0.22,...,66817.92,28008.00,41955.44,49249.32,30647.43,38985.31,NaN,NaN,NaN,NaN
3,846_0.pkl,846,0,8375,0,100.00,Entertainment/public assembly,-0.43,-0.47,-0.41,...,26.00,3.00,12.00,15.49,7.81,12.13,mean,mean,NaN,NaN
4,816_0.pkl,816,0,7752,0,100.00,Entertainment/public assembly,-0.35,-0.38,-0.35,...,118.00,0.00,84.87,202.00,0.00,90.03,mean,mean,mean,mean
5,693_0.pkl,693,0,8784,0,100.00,Office,-0.15,-0.13,-0.16,...,3109.30,5.02,2170.50,2864.03,-125.76,1918.28,mean,mean,mean,mean
6,621_0.pkl,621,0,3062,0,100.00,Parking,0.02,0.12,-0.01,...,452.61,289.63,391.78,467.00,276.31,377.19,mean,mean,mean,mean
7,375_0.pkl,375,0,8782,0,100.00,Office,0.14,0.21,0.10,...,60541.68,42270.58,50942.46,60639.82,40470.00,50331.05,NaN,NaN,NaN,NaN
8,144_0.pkl,144,0,8784,0,100.00,Office,-0.53,-0.57,-0.52,...,1976.75,1262.25,1664.78,1900.45,1334.51,1621.95,mean,mean,mean,mean
9,1148_2.pkl,1148,2,8784,2,99.98,Office,-0.88,-0.86,-0.88,...,1769742.40,391546.80,1260762.54,1717172.50,538519.62,1281346.38,NaN,NaN,NaN,NaN


In [91]:
df= df_results.filter(regex='RMSE')
df.head()

,Model_1_RMSE,Model_2_RMSE,Model_3_RMSE,Model_4_RMSE,Model_5_RMSE
0,2571.74,3189.77,13842.85,3223.88,4735.66
1,309.76,403.18,560.34,293.97,290.77
2,14556.08,13360.91,26502.04,11803.46,10160.78
3,3.15,4.57,9.68,3.59,3.38
4,33.41,31.65,84.29,29.42,31.13


## Time Series Split Cross Validation

In [ ]:
tss =  TimeSeriesSplit(n_splits=4,gap=4,test_size=14,max_train_size=120) #too generic
rmse = []
count = 1

results = pd.DataFrame()


# Write function that rolls next pd.delta?

for train_index, test_index in tss.split(df_train): 
    
    cv_train, cv_test = df_train.iloc[train_index]['meter_reading'], df_train.iloc[test_index]['meter_reading']
    model = pm.auto_arima(cv_train,trace=True,n_fits=20)
    print(model.summary())
    
    # 
    predicts = model.predict(n_periods=14)
    true_values = cv_test.values
    error = ((sqrt(mean_squared_error(predicts,true_values))))
    model.plot_diagnostics()
    rmse.append(error)
    print(error)
print('RMSE')
print(np.mean(rmse))

In [ ]:
df_tyra
print(model.summary())

# 
predicts = model.predict(n_periods=14)
true_values = cv_test.values
error = ((sqrt(mean_squared_error(predicts,true_values))))
model.plot_diagnostics()
rmse.append(error)
print(error)

In [ ]:
train.plot()

In [ ]:
test.plot()

In [ ]:
model.predict(n_periods=20)

In [ ]:

#                      start_p=1,max_p=7,
#                      start_q=1,max_q=7,
#                      start_P=1,max_P=7,
#                      start_Q=1,max_Q=7,
preds = model.predict(n_periods=test.shape[0], return_conf_int=False)

# cv = model_selection.SlidingWindowForecastCV(window_size=3,step=1,h=4)
# predictions = model_selection.cross_val_predict(model, test, cv=cv, verbose=2, averaging='median')
# # temp_model_cv_scores = model_selection.cross_val_score(model, train, scoring='smape', cv=cv, verbose=2)

# pred = pd.Series(predictions)
# df_temp_preds = pd.DataFrame(pred,index=test.index,columns=['predictions'])
# df_preds =pd.concat([df_temp_preds,test],axis=1)
# df_preds

# Print the error:
print("Test RMSE: %.3f" % np.sqrt(mean_squared_error(test, preds)))


# Plot Forecasts
x_axis = np.arange(train.shape[0] + preds.shape[0])
x_years = x_axis + 2016  # Year starts at 1821

plt.plot(preds,kind='line')

In [ ]:

model.plot_diagnostics()

In [ ]:
model

In [ ]:
preds

In [ ]:
model

In [ ]:
preds

In [ ]:
df_temp_preds = pd.DataFrame(preds,index=test.index,columns=['predictions'])
df_preds =pd.concat([df_temp_preds,test],axis=1)
df_preds

In [ ]:
# Plot Forecasts
x_axis = np.arange(train.shape[0] + preds.shape[0])
x_years = x_axis + 2016  # Year starts at 1821

plt.plot(df_preds,kind='line')

# plt.fill_between(x_years[x_axis[-preds.shape[0]:]],
#                  conf_int[:, 0], conf_int[:, 1],
#                  alpha=0.1, color='b')
# plt.title("Temperature Forecast")
# plt.xlabel("Year")

# ARIMA

### Set PDQ

# Rolling Forecast ARIMA
### Walk-forward validation

### Grid Search ARIMA / Evaluation

# Facebook Prophet
## Additive Time Series Forecasting Model

In [ ]:
from fbprophet import Prophet

In [ ]:
data_len = 366
split_test_doy = 270

df_train = df.iloc[0:split_test_doy]
df_test = df.iloc[split_test_doy:data_len]

In [ ]:
df_prophet_train = df_train.reset_index()
df_prophet_train = df_prophet_train[['timestamp','meter_reading']]
df_prophet_train = df_prophet_train.rename(columns={'timestamp':'ds','meter_reading':'y'})

df_prophet_test = df_test.reset_index()
df_prophet_test = df_prophet_test[['timestamp','meter_reading']]
df_prophet_test = df_prophet_test.rename(columns={'timestamp':'ds','meter_reading':'y'})

df_prophet_predict = df_test.reset_index()
df_prophet_predict = df_prophet_predict[['timestamp']]
df_prophet_predict = df_prophet_predict.rename(columns={'timestamp':'ds'})


In [ ]:
model = Prophet()
model.fit(df_prophet_train)

In [ ]:
df_forecast = model.predict(df_prophet_predict)

In [ ]:
print(df_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

In [ ]:
model.plot(forecast)